<a href="https://colab.research.google.com/github/lfenjoy9/notes/blob/master/notebook1_ball_detection_signals_v0_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 1 – Ball → Motion → Playing Segments → Video Clips**  
**Version: v0_18**

This notebook processes a table-tennis match video and automatically extracts **only the playing clips**.

Key design in this version:
- **Window-based static detection** so repeated serves at the same spot are *not* incorrectly treated as static.
- Save all YOLO detections to `ball_detections_raw.csv`, then build a per-frame `ball_signals.csv` after filtering static detections.
- **Fast batched YOLO prediction with sequential frame streaming** (no per-frame `cap.set` seeks).
- Cut playing clips and non-playing clips, both **restricted to the analyzed time window** (up to `MAX_DURATION_SECONDS`).

Debug images (optional) show:
- All detections in each frame (stage 1, yellow),
- Detections treated as static (stage 2, red with `STATIC` label),
- The chosen non-static ball detection per frame (green).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q ultralytics opencv-python pandas tqdm numpy

from ultralytics import YOLO
from pathlib import Path
import cv2
import pandas as pd
import numpy as np
import json
import os
import shutil
import time
from tqdm.auto import tqdm

VERSION = 'v0_18'
print('Notebook version:', VERSION)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Notebook version: v0_18


In [ ]:
# ====== CONFIG ======
# ORIGINAL_VIDEO_PATH = Path('/content/drive/MyDrive/Science Fair/Demo/1208-60-batch/1208.mp4')
ORIGINAL_VIDEO_PATH = Path('/content/drive/MyDrive/Science Fair/Demo/x/2024_national_IMG 7269~video [U0vyhSW3Isw].mp4')
ORIGINAL_MODEL_PATH = Path('/content/drive/MyDrive/Science Fair/Models/2025 Science Fair.v11i.yolov8.zip_yolov8s_best.pt')

ORIGINAL_VIDEO_DIR = ORIGINAL_VIDEO_PATH.parent

LOCAL_VIDEO_PATH = Path('/content/input_video.mp4')
LOCAL_MODEL_PATH = Path('/content/model.pt')

OUTPUT_DIR = Path('/content/ball_signals')

BALL_CLASS_IDS = {0}
CONF_THRES = 0.10

# Inference control
MAX_DURATION_SECONDS = None      # only analyze from the start up to this time window
FRAME_STRIDE = 1               # process every N-th frame (1 = every frame)
BATCH_SIZE =  128                # YOLO batch size for prediction
# Recommended BATCH_SIZE:
# - CPU only:            1–2 (start with 1)
# - Colab T4 / L4:       8–16 (start with 8)
# - V100 / A100 / 16GB+: 16–32 (start with 16)

# If this file exists on Drive, reuse it instead of rerunning detection
BALL_CSV_DRIVE_PATH = ORIGINAL_VIDEO_DIR / 'ball_signals.csv'

DEBUG_SAVE_ANNOTATED = False
ANNOTATED_DIR = OUTPUT_DIR / 'annotated_frames'

MOVE_DIST_THRESH = 5.0

# Static-region detection (window-based)
STATIC_GRID_SIZE = 16          # pixels
STATIC_WINDOW_SEC = 0.5        # seconds
STATIC_WINDOW_MIN_COUNT = 5    # detections in window to call static

# Playing-segment parameters
CONF_PLAY_THRESH = 0.5
MIN_CLIP_DURATION_SEC = 1.0
MAX_GAP_INSIDE_SEC = 0.5
PAD_BEFORE_SEC = 0.2
PAD_AFTER_SEC = 0.3

EXTRA_CUT_BUFFER_SEC = 0.25
COMBINE_PLAYING_CLIPS = True
COMBINE_NONPLAYING_CLIPS = True
MIN_NONPLAY_DURATION_SEC = 0.5

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
if DEBUG_SAVE_ANNOTATED:
    ANNOTATED_DIR.mkdir(parents=True, exist_ok=True)

print('Original video path:', ORIGINAL_VIDEO_PATH)
print('Original model path:', ORIGINAL_MODEL_PATH)
print('Original video dir:', ORIGINAL_VIDEO_DIR)
print('BALL_CSV_DRIVE_PATH:', BALL_CSV_DRIVE_PATH)
print('FRAME_STRIDE:', FRAME_STRIDE)
print('BATCH_SIZE:', BATCH_SIZE)
print('STATIC_GRID_SIZE:', STATIC_GRID_SIZE,
      'STATIC_WINDOW_SEC:', STATIC_WINDOW_SEC,
      'STATIC_WINDOW_MIN_COUNT:', STATIC_WINDOW_MIN_COUNT)

Original video path: /content/drive/MyDrive/Science Fair/Demo/x/2024_national_IMG 7269~video [U0vyhSW3Isw].mp4
Original model path: /content/drive/MyDrive/Science Fair/Models/2025 Science Fair.v11i.yolov8.zip_yolov8s_best.pt
Original video dir: /content/drive/MyDrive/Science Fair/Demo/x
BALL_CSV_DRIVE_PATH: /content/drive/MyDrive/Science Fair/Demo/x/ball_signals.csv
FRAME_STRIDE: 1
BATCH_SIZE: 128
STATIC_GRID_SIZE: 16 STATIC_WINDOW_SEC: 0.5 STATIC_WINDOW_MIN_COUNT: 5


In [ ]:
# ====== COPY TO LOCAL & LOAD VIDEO / MODEL (for metadata) ======
assert ORIGINAL_VIDEO_PATH.exists(), f'Video not found: {ORIGINAL_VIDEO_PATH}'
assert ORIGINAL_MODEL_PATH.exists(), f'Model not found: {ORIGINAL_MODEL_PATH}'

if not LOCAL_VIDEO_PATH.exists():
    print('Copying video to runtime...')
    shutil.copy2(ORIGINAL_VIDEO_PATH, LOCAL_VIDEO_PATH)
if not LOCAL_MODEL_PATH.exists():
    print('Copying model to runtime...')
    shutil.copy2(ORIGINAL_MODEL_PATH, LOCAL_MODEL_PATH)

VIDEO_PATH = LOCAL_VIDEO_PATH
MODEL_PATH = LOCAL_MODEL_PATH

print('Using VIDEO_PATH =', VIDEO_PATH)
print('Using MODEL_PATH =', MODEL_PATH)

print('Loading YOLO model...')
model = YOLO(str(MODEL_PATH))

# Open once just to read metadata
cap_meta = cv2.VideoCapture(str(VIDEO_PATH))
if not cap_meta.isOpened():
    raise RuntimeError(f'Failed to open video for metadata: {VIDEO_PATH}')

fps = cap_meta.get(cv2.CAP_PROP_FPS) or 0.0
frame_count = int(cap_meta.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap_meta.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap_meta.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap_meta.release()

print('FPS:', fps)
print('Total frame count:', frame_count)
print('Resolution:', width, 'x', height)

if fps > 0 and MAX_DURATION_SECONDS is not None:
    max_frames_by_time = int(MAX_DURATION_SECONDS * fps)
    MAX_FRAMES_TO_PROCESS = min(frame_count, max_frames_by_time)
else:
    MAX_FRAMES_TO_PROCESS = frame_count

approx_seconds = MAX_FRAMES_TO_PROCESS / fps if fps > 0 else 0.0
frame_indices = list(range(0, MAX_FRAMES_TO_PROCESS, FRAME_STRIDE))

print('MAX_FRAMES_TO_PROCESS:', MAX_FRAMES_TO_PROCESS)
print('Approx duration covered (s):', approx_seconds)
print('Detection steps (len(frame_indices)):', len(frame_indices))

Copying video to runtime...
Copying model to runtime...
Using VIDEO_PATH = /content/input_video.mp4
Using MODEL_PATH = /content/model.pt
Loading YOLO model...
FPS: 29.97002997002997
Total frame count: 4894
Resolution: 1920 x 1080
MAX_FRAMES_TO_PROCESS: 4894
Approx duration covered (s): 163.29646666666667
Detection steps (len(frame_indices)): 4894


In [ ]:
# ====== DETECTION: REUSE ball_signals.csv OR RUN YOLO (BATCHED, STREAMING) ======
signals_csv_local = OUTPUT_DIR / 'ball_signals.csv'
raw_csv_local = OUTPUT_DIR / 'ball_detections_raw.csv'
annotated_zip_path = None

ran_detection = False
df_raw = None

if BALL_CSV_DRIVE_PATH.exists():
    print('Reusing existing ball_signals.csv from Drive:', BALL_CSV_DRIVE_PATH)
    shutil.copy2(BALL_CSV_DRIVE_PATH, signals_csv_local)
    print('Copied ball_signals.csv to local runtime:', signals_csv_local)
else:
    print('ball_signals.csv not found on Drive; running YOLO ball detection (batched, streaming)...')
    ran_detection = True

    raw_dets = []

    if BATCH_SIZE is None or BATCH_SIZE <= 0:
        BATCH_SIZE = 1

    start_time = time.time()

    # Open a fresh VideoCapture for streaming
    cap_stream = cv2.VideoCapture(str(VIDEO_PATH))
    if not cap_stream.isOpened():
        raise RuntimeError(f'Failed to open video for detection: {VIDEO_PATH}')

    needed_indices = set(frame_indices)
    frames_batch = []
    idx_batch = []
    times_batch = []

    for cur in tqdm(range(MAX_FRAMES_TO_PROCESS), desc=f'Detecting ball (stride={FRAME_STRIDE}, batch={BATCH_SIZE})'):
        ret, frame = cap_stream.read()
        if not ret:
            print('Failed to read frame', cur, '- stopping early.')
            break

        if cur in needed_indices:
            frames_batch.append(frame)
            idx_batch.append(cur)
            time_sec = cur / fps if fps > 0 else 0.0
            times_batch.append(time_sec)

            if len(frames_batch) >= BATCH_SIZE:
                results_list = model.predict(frames_batch, conf=CONF_THRES, verbose=False)
                for frame_idx, t_sec, result in zip(idx_batch, times_batch, results_list):
                    boxes = result.boxes
                    if boxes is None or boxes.cls is None:
                        continue
                    for i in range(len(boxes.cls)):
                        cls_id = int(boxes.cls[i])
                        conf_val = float(boxes.conf[i])
                        if cls_id in BALL_CLASS_IDS:
                            x1, y1, x2, y2 = boxes.xyxy[i].tolist()
                            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                            cx = (x1 + x2) / 2.0
                            cy = (y1 + y2) / 2.0
                            raw_dets.append({
                                'frame_idx': frame_idx,
                                'time_sec': t_sec,
                                'cls_id': cls_id,
                                'conf': conf_val,
                                'x1': x1,
                                'y1': y1,
                                'x2': x2,
                                'y2': y2,
                                'center_x': cx,
                                'center_y': cy,
                            })
                frames_batch = []
                idx_batch = []
                times_batch = []

    # Flush remaining batch
    if frames_batch:
        results_list = model.predict(frames_batch, conf=CONF_THRES, verbose=False)
        for frame_idx, t_sec, result in zip(idx_batch, times_batch, results_list):
            boxes = result.boxes
            if boxes is None or boxes.cls is None:
                continue
            for i in range(len(boxes.cls)):
                cls_id = int(boxes.cls[i])
                conf_val = float(boxes.conf[i])
                if cls_id in BALL_CLASS_IDS:
                    x1, y1, x2, y2 = boxes.xyxy[i].tolist()
                    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                    cx = (x1 + x2) / 2.0
                    cy = (y1 + y2) / 2.0
                    raw_dets.append({
                        'frame_idx': frame_idx,
                        'time_sec': t_sec,
                        'cls_id': cls_id,
                        'conf': conf_val,
                        'x1': x1,
                        'y1': y1,
                        'x2': x2,
                        'y2': y2,
                        'center_x': cx,
                        'center_y': cy,
                    })

    cap_stream.release()

    end_time = time.time()
    elapsed = end_time - start_time
    print('Finished detection loop.')
    print('Total detection steps (len(frame_indices)):', len(frame_indices))
    if len(frame_indices) > 0:
        print(f'Avg time per detected frame: {elapsed / len(frame_indices):.4f} s')

    df_raw = pd.DataFrame(raw_dets)
    df_raw.to_csv(raw_csv_local, index=False)
    print('Saved raw detections CSV to:', raw_csv_local)

    raw_csv_drive = ORIGINAL_VIDEO_DIR / raw_csv_local.name
    try:
        shutil.copy2(raw_csv_local, raw_csv_drive)
        print('Copied raw detections CSV to Drive:', raw_csv_drive)
    except Exception as e:
        print('Failed to copy raw detections CSV to Drive:', e)

    # ---- Static-region detection (window-based) ----
    static_cells = set()
    static_indices = set()

    if df_raw is not None and not df_raw.empty and fps > 0:
        df_raw['grid_x'] = (df_raw['center_x'] // STATIC_GRID_SIZE).astype(int)
        df_raw['grid_y'] = (df_raw['center_y'] // STATIC_GRID_SIZE).astype(int)

        frames_per_window = max(1, int(STATIC_WINDOW_SEC * fps))
        print('STATIC_WINDOW_SEC:', STATIC_WINDOW_SEC,
              'frames_per_window:', frames_per_window)
        print('STATIC_WINDOW_MIN_COUNT:', STATIC_WINDOW_MIN_COUNT)

        for (gx, gy), grp in df_raw.groupby(['grid_x', 'grid_y']):
            frame_list = sorted(grp['frame_idx'].unique())
            if len(frame_list) < STATIC_WINDOW_MIN_COUNT:
                continue

            i = 0
            j = 0
            n = len(frame_list)

            while i < n:
                while j < n and frame_list[j] - frame_list[i] <= frames_per_window:
                    j += 1
                window_count = j - i

                if window_count >= STATIC_WINDOW_MIN_COUNT:
                    static_cells.add((gx, gy))
                    start_f = frame_list[i]
                    end_f = frame_list[j - 1]

                    mask = (
                        (df_raw['grid_x'] == gx) &
                        (df_raw['grid_y'] == gy) &
                        (df_raw['frame_idx'] >= start_f) &
                        (df_raw['frame_idx'] <= end_f)
                    )
                    static_indices.update(df_raw[mask].index.tolist())

                i += 1

        print('Number of static cells (window-based):', len(static_cells))
        print('Number of static detections (rows):', len(static_indices))

        df_raw['is_static'] = 0
        if static_indices:
            df_raw.loc[list(static_indices), 'is_static'] = 1
    else:
        print('No raw detections or invalid fps -> no static cells.')
        if df_raw is not None:
            df_raw['is_static'] = 0

    # ---- Build per-frame summary (ball_signals) with static filtering ----
    frame_rows = []

    if df_raw is None or df_raw.empty:
        for frame_idx in frame_indices:
            time_sec = frame_idx / fps if fps > 0 else 0.0
            frame_rows.append({
                'frame_idx': frame_idx,
                'time_sec': time_sec,
                'has_ball': 0,
                'num_ball': 0,
                'max_conf': 0.0,
                'bbox_x1': None,
                'bbox_y1': None,
                'bbox_x2': None,
                'bbox_y2': None,
                'num_raw': 0,
                'num_filtered': 0,
            })
    else:
        grouped = df_raw.groupby('frame_idx')

        for frame_idx in frame_indices:
            time_sec = frame_idx / fps if fps > 0 else 0.0

            if frame_idx not in grouped.groups:
                frame_rows.append({
                    'frame_idx': frame_idx,
                    'time_sec': time_sec,
                    'has_ball': 0,
                    'num_ball': 0,
                    'max_conf': 0.0,
                    'bbox_x1': None,
                    'bbox_y1': None,
                    'bbox_x2': None,
                    'bbox_y2': None,
                    'num_raw': 0,
                    'num_filtered': 0,
                })
                continue

            df_f = grouped.get_group(frame_idx).copy()
            num_raw = len(df_f)

            if 'is_static' not in df_f.columns:
                df_f['is_static'] = 0

            df_filtered = df_f[df_f['is_static'] == 0]
            num_filtered = len(df_filtered)

            if num_filtered == 0:
                frame_rows.append({
                    'frame_idx': frame_idx,
                    'time_sec': time_sec,
                    'has_ball': 0,
                    'num_ball': 0,
                    'max_conf': 0.0,
                    'bbox_x1': None,
                    'bbox_y1': None,
                    'bbox_x2': None,
                    'bbox_y2': None,
                    'num_raw': num_raw,
                    'num_filtered': 0,
                })
                continue

            best = df_filtered.loc[df_filtered['conf'].idxmax()]

            frame_rows.append({
                'frame_idx': frame_idx,
                'time_sec': time_sec,
                'has_ball': 1,
                'num_ball': num_filtered,
                'max_conf': float(best['conf']),
                'bbox_x1': int(best['x1']),
                'bbox_y1': int(best['y1']),
                'bbox_x2': int(best['x2']),
                'bbox_y2': int(best['y2']),
                'num_raw': num_raw,
                'num_filtered': num_filtered,
            })

    df_signals = pd.DataFrame(frame_rows)
    df_signals.to_csv(signals_csv_local, index=False)
    print('Saved ball_signals.csv to:', signals_csv_local)

    try:
        shutil.copy2(signals_csv_local, BALL_CSV_DRIVE_PATH)
        print('Copied ball_signals.csv to Drive:', BALL_CSV_DRIVE_PATH)
    except Exception as e:
        print('Failed to copy ball_signals.csv to Drive:', e)

    # ---- Debug annotated frames (Stage 1 + Stage 2) ----
    if DEBUG_SAVE_ANNOTATED and df_raw is not None and not df_raw.empty:
        ANNOTATED_DIR.mkdir(parents=True, exist_ok=True)
        print('Generating annotated frames...')

        cap_anno = cv2.VideoCapture(str(VIDEO_PATH))
        df_signals_idx = df_signals.set_index('frame_idx')
        grouped_raw = df_raw.groupby('frame_idx')

        for frame_idx in tqdm(frame_indices, desc='Annotating frames'):
            cap_anno.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            ret, frame = cap_anno.read()
            if not ret:
                break

            draw_frame = frame.copy()
            time_sec = frame_idx / fps if fps > 0 else 0.0

            if frame_idx in grouped_raw.groups:
                dets = grouped_raw.get_group(frame_idx)
                for _, det in dets.iterrows():
                    x1, y1, x2, y2 = int(det['x1']), int(det['y1']), int(det['x2']), int(det['y2'])
                    conf_val = float(det['conf'])
                    is_static = int(det.get('is_static', 0))

                    cv2.rectangle(draw_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                    cv2.putText(draw_frame, f"{conf_val:.2f}", (x1, max(0, y1 - 8)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)

                    if is_static == 1:
                        cv2.rectangle(draw_frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
                        cv2.putText(draw_frame, 'STATIC', (x1, max(0, y1 - 24)),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)

            if frame_idx in df_signals_idx.index:
                row_sig = df_signals_idx.loc[frame_idx]
                has_ball = int(row_sig['has_ball'])
                num_ball = int(row_sig['num_ball'])
                max_conf = float(row_sig['max_conf'])

                if has_ball == 1 and not pd.isna(row_sig['bbox_x1']):
                    bx1 = int(row_sig['bbox_x1'])
                    by1 = int(row_sig['bbox_y1'])
                    bx2 = int(row_sig['bbox_x2'])
                    by2 = int(row_sig['bbox_y2'])
                    cv2.rectangle(draw_frame, (bx1, by1), (bx2, by2), (0, 255, 0), 3)

                if fps > 0:
                    time_txt = f"frame: {frame_idx} ({time_sec:.2f}s)"
                else:
                    time_txt = f"frame: {frame_idx}"
                balls_txt = f"balls: {num_ball}"
                conf_txt = f"max conf: {max_conf:.2f}"

                if num_ball == 0:
                    text_color = (0, 0, 255)
                elif num_ball > 1:
                    text_color = (0, 255, 255)
                else:
                    if max_conf < 0.5:
                        text_color = (255, 0, 255)
                    else:
                        text_color = (0, 255, 0)

                h, w = draw_frame.shape[:2]
                margin = 10
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 1.5
                thickness = 3

                lines = [time_txt, balls_txt, conf_txt]
                x = w - margin
                y = margin + int(30 * font_scale)

                for tline in lines:
                    (tw, th), _ = cv2.getTextSize(tline, font, font_scale, thickness)
                    tx = x - tw
                    ty = y
                    cv2.putText(draw_frame, tline, (tx, ty), font, font_scale,
                                text_color, thickness, cv2.LINE_AA)
                    y += th + 12

            out_path = ANNOTATED_DIR / f'frame_{frame_idx:06d}.jpg'
            cv2.imwrite(str(out_path), draw_frame)

        cap_anno.release()

        zip_base = OUTPUT_DIR / 'annotated_frames'
        shutil.make_archive(str(zip_base), 'zip', root_dir=ANNOTATED_DIR)
        annotated_zip_path = str(zip_base) + '.zip'
        print('Annotated frames zipped to:', annotated_zip_path)

        dest_path = ORIGINAL_VIDEO_DIR / Path(annotated_zip_path).name
        try:
            shutil.copy2(annotated_zip_path, dest_path)
            print('Copied annotated zip to Google Drive:', dest_path)
        except Exception as e:
            print('Failed to copy annotated zip to Google Drive:', e)

print('Final local ball_signals CSV path:', signals_csv_local)
if os.path.exists(raw_csv_local):
    print('Final local raw detections CSV path:', raw_csv_local)

ball_signals.csv not found on Drive; running YOLO ball detection (batched, streaming)...


Detecting ball (stride=1, batch=128):   0%|          | 0/4894 [00:00<?, ?it/s]

Finished detection loop.
Total detection steps (len(frame_indices)): 4894
Avg time per detected frame: 0.0137 s
Saved raw detections CSV to: /content/ball_signals/ball_detections_raw.csv
Copied raw detections CSV to Drive: /content/drive/MyDrive/Science Fair/Demo/x/ball_detections_raw.csv
STATIC_WINDOW_SEC: 0.5 frames_per_window: 14
STATIC_WINDOW_MIN_COUNT: 5
Number of static cells (window-based): 12
Number of static detections (rows): 204
Saved ball_signals.csv to: /content/ball_signals/ball_signals.csv
Copied ball_signals.csv to Drive: /content/drive/MyDrive/Science Fair/Demo/x/ball_signals.csv
Final local ball_signals CSV path: /content/ball_signals/ball_signals.csv
Final local raw detections CSV path: /content/ball_signals/ball_detections_raw.csv


In [ ]:
# ====== LOAD ball_signals.csv AND COMPUTE MOTION ======
assert signals_csv_local.exists(), f'Local ball_signals CSV not found: {signals_csv_local}'
df = pd.read_csv(signals_csv_local)
print('Loaded ball_signals CSV with columns:', list(df.columns))

df_motion = df.copy()

df_motion['center_x'] = (df_motion['bbox_x1'] + df_motion['bbox_x2']) / 2.0
df_motion['center_y'] = (df_motion['bbox_y1'] + df_motion['bbox_y2']) / 2.0

df_motion['prev_center_x'] = df_motion['center_x'].shift(1)
df_motion['prev_center_y'] = df_motion['center_y'].shift(1)
df_motion['next_center_x'] = df_motion['center_x'].shift(-1)
df_motion['next_center_y'] = df_motion['center_y'].shift(-1)

dx_prev = df_motion['center_x'] - df_motion['prev_center_x']
dy_prev = df_motion['center_y'] - df_motion['prev_center_y']
df_motion['dist_prev'] = np.sqrt(dx_prev**2 + dy_prev**2)

dx_next = df_motion['center_x'] - df_motion['next_center_x']
dy_next = df_motion['center_y'] - df_motion['next_center_y']
df_motion['dist_next'] = np.sqrt(dx_next**2 + dy_next**2)

df_motion['is_moving'] = (
    (df_motion['has_ball'] == 1) &
    (df_motion['dist_prev'] > MOVE_DIST_THRESH) &
    (df_motion['dist_next'] > MOVE_DIST_THRESH)
).astype(int)

print('Motion columns added.')
df_motion.head(10)

Loaded ball_signals CSV with columns: ['frame_idx', 'time_sec', 'has_ball', 'num_ball', 'max_conf', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'num_raw', 'num_filtered']
Motion columns added.


,frame_idx,time_sec,has_ball,num_ball,max_conf,bbox_x1,bbox_y1,bbox_x2,bbox_y2,num_raw,num_filtered,center_x,center_y,prev_center_x,prev_center_y,next_center_x,next_center_y,dist_prev,dist_next,is_moving
0,0,0.000000,1,1,0.567938,547.0,378.0,559.0,390.0,1,1,553.0,384.0,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,0.033367,0,0,0.000000,NaN,NaN,NaN,NaN,1,0,NaN,NaN,553.0,384.0,NaN,NaN,NaN,NaN,0
2,2,0.066733,0,0,0.000000,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,553.5,366.5,NaN,NaN,0
3,3,0.100100,1,1,0.702808,548.0,361.0,559.0,372.0,1,1,553.5,366.5,NaN,NaN,553.5,366.5,NaN,0.0,0
4,4,0.133467,1,1,0.511492,548.0,361.0,559.0,372.0,1,1,553.5,366.5,553.5,366.5,NaN,NaN,0.0,NaN,0
5,5,0.166833,0,0,0.000000,NaN,NaN,NaN,NaN,1,0,NaN,NaN,553.5,366.5,NaN,NaN,NaN,NaN,0
6,6,0.200200,0,0,0.000000,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,7,0.233567,0,0,0.000000,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,8,0.266933,0,0,0.000000,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,9,0.300300,0,0,0.000000,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
# ====== SAVE MOTION CSV (LOCAL + DRIVE) ======
motion_csv_local = OUTPUT_DIR / 'ball_motion_signals.csv'
df_motion.to_csv(motion_csv_local, index=False)
print('Saved motion CSV locally to:', motion_csv_local)

motion_csv_drive = ORIGINAL_VIDEO_DIR / motion_csv_local.name
try:
    shutil.copy2(motion_csv_local, motion_csv_drive)
    print('Copied motion CSV to Google Drive:', motion_csv_drive)
except Exception as e:
    print('Failed to copy motion CSV to Google Drive:', e)

Saved motion CSV locally to: /content/ball_signals/ball_motion_signals.csv
Copied motion CSV to Google Drive: /content/drive/MyDrive/Science Fair/Demo/x/ball_motion_signals.csv


In [ ]:
# ====== COMPUTE PLAYING SEGMENTS ======
if fps <= 0:
    raise RuntimeError('FPS must be > 0 to compute segments.')

MAX_GAP_INSIDE_FRAMES = int(MAX_GAP_INSIDE_SEC * fps)
print('MAX_GAP_INSIDE_FRAMES:', MAX_GAP_INSIDE_FRAMES)

df_motion['is_playing_core'] = (
    (df_motion['has_ball'] == 1) &
    (df_motion['max_conf'] >= CONF_PLAY_THRESH) &
    (df_motion['is_moving'] == 1)
).astype(int)

flags = df_motion['is_playing_core'].values
frames = df_motion['frame_idx'].values

segments = []
in_seg = False
seg_start = None
last_play_frame = None
gap_len = 0

for f, flag in zip(frames, flags):
    if flag == 1:
        if not in_seg:
            in_seg = True
            seg_start = f
        gap_len = 0
        last_play_frame = f
    else:
        if in_seg:
            gap_len += 1
            if gap_len > MAX_GAP_INSIDE_FRAMES:
                seg_end = last_play_frame if last_play_frame is not None else f - gap_len
                segments.append((seg_start, seg_end))
                in_seg = False
                seg_start = None
                last_play_frame = None
                gap_len = 0

if in_seg and last_play_frame is not None:
    segments.append((seg_start, last_play_frame))

print('Raw playing segments (frame idx):', segments)

filtered_segments = []
for (s, e) in segments:
    dur = (e - s + 1) / fps
    if dur >= MIN_CLIP_DURATION_SEC:
        filtered_segments.append((s, e))

print('Segments after min-duration filter:', filtered_segments)

clip_rows = []
max_time_sec = df_motion['time_sec'].max() if 'time_sec' in df_motion.columns else (frame_count / fps)

for cid, (s, e) in enumerate(filtered_segments, start=1):
    start_sec = max(0.0, s / fps - PAD_BEFORE_SEC)
    end_sec = min(max_time_sec, e / fps + PAD_AFTER_SEC)
    duration_sec = max(0.0, end_sec - start_sec)

    clip_rows.append({
        'clip_id': cid,
        'start_frame': int(s),
        'end_frame': int(e),
        'start_sec': float(start_sec),
        'end_sec': float(end_sec),
        'duration_sec': float(duration_sec),
    })

clips_df = pd.DataFrame(clip_rows)
playing_clips_csv_local = OUTPUT_DIR / 'playing_clips.csv'
clips_df.to_csv(playing_clips_csv_local, index=False)
print('Saved playing_clips.csv to:', playing_clips_csv_local)

playing_clips_csv_drive = ORIGINAL_VIDEO_DIR / playing_clips_csv_local.name
try:
    shutil.copy2(playing_clips_csv_local, playing_clips_csv_drive)
    print('Copied playing_clips.csv to Drive:', playing_clips_csv_drive)
except Exception as e:
    print('Failed to copy playing_clips.csv to Drive:', e)

clips_df.head()

MAX_GAP_INSIDE_FRAMES: 14
Raw playing segments (frame idx): [(np.int64(49), np.int64(100)), (np.int64(178), np.int64(186)), (np.int64(241), np.int64(299)), (np.int64(316), np.int64(336)), (np.int64(454), np.int64(491)), (np.int64(562), np.int64(608)), (np.int64(628), np.int64(631)), (np.int64(806), np.int64(831)), (np.int64(899), np.int64(948)), (np.int64(1036), np.int64(1073)), (np.int64(1101), np.int64(1110)), (np.int64(1171), np.int64(1246)), (np.int64(1391), np.int64(1414)), (np.int64(1463), np.int64(1495)), (np.int64(1512), np.int64(1547)), (np.int64(1858), np.int64(1924)), (np.int64(2209), np.int64(2241)), (np.int64(2321), np.int64(2377)), (np.int64(2437), np.int64(2552)), (np.int64(2719), np.int64(2733)), (np.int64(2788), np.int64(2963)), (np.int64(3066), np.int64(3073)), (np.int64(3105), np.int64(3136)), (np.int64(3340), np.int64(3421)), (np.int64(3441), np.int64(3446)), (np.int64(3562), np.int64(3592)), (np.int64(3670), np.int64(3706)), (np.int64(3736), np.int64(3736)), (np.in

,clip_id,start_frame,end_frame,start_sec,end_sec,duration_sec
0,1,49,100,1.434967,3.636667,2.201700
1,2,241,299,7.841367,10.276633,2.435267
2,3,454,491,14.948467,16.683033,1.734567
3,4,562,608,18.552067,20.586933,2.034867
4,5,899,948,29.796633,31.931600,2.134967


In [ ]:
# ====== CUT VIDEO INTO CLIPS & COMBINED VIDEOS ======
video_duration_sec = frame_count / fps if fps > 0 else None
analysis_end_sec = max_time_sec  # limit both playing + non-playing to analyzed window
print('Video duration (sec):', video_duration_sec)
print('Analysis end (sec):', analysis_end_sec)

CLIPS_OUTPUT_DIR = OUTPUT_DIR / 'playing_clips_videos'
CLIPS_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DRIVE_CLIPS_DIR = ORIGINAL_VIDEO_DIR / 'playing_clips_videos'
try:
    DRIVE_CLIPS_DIR.mkdir(parents=True, exist_ok=True)
except Exception as e:
    print('Could not create Drive clips dir:', e)

playing_clip_paths = []

if clips_df.empty:
    print('No playing clips detected; skipping cutting.')
else:
    os.system('ffmpeg -version | head -n 1')

    for _, row in clips_df.iterrows():
        cid = int(row['clip_id'])
        s = float(row['start_sec'])
        e = float(row['end_sec'])

        # Restrict to analysis window
        s = max(0.0, min(s, analysis_end_sec))
        e = max(0.0, min(e, analysis_end_sec))

        start = max(0.0, s - EXTRA_CUT_BUFFER_SEC)
        end = e + EXTRA_CUT_BUFFER_SEC

        # Also don't go beyond analysis_end_sec when actually cutting
        end = min(analysis_end_sec, end)
        if end <= start:
            continue

        duration = end - start
        out_name = f'clip_{cid:03d}_{start:06.2f}-{end:06.2f}.mp4'
        out_path = CLIPS_OUTPUT_DIR / out_name

        cmd = (
            f"ffmpeg -y -ss {start:.3f} -i '{VIDEO_PATH}' "
            f"-t {duration:.3f} -c:v libx264 -preset veryfast -crf 18 "
            f"-c:a aac -movflags +faststart '{out_path}'"
        )
        print('Cutting playing clip', cid, 'start=', start, 'end=', end)
        os.system(cmd)

        if out_path.exists():
            playing_clip_paths.append(out_path)
            try:
                shutil.copy2(out_path, DRIVE_CLIPS_DIR / out_name)
            except Exception as e:
                print('Failed to copy playing clip to Drive:', e)

combined_playing_local = None
combined_playing_drive = None

if COMBINE_PLAYING_CLIPS and playing_clip_paths:
    concat_txt = CLIPS_OUTPUT_DIR / 'playing_concat_list.txt'
    with open(concat_txt, 'w') as f:
        for p in playing_clip_paths:
            f.write(f"file '{p}'\n")

    combined_playing_local = CLIPS_OUTPUT_DIR / 'all_playing.mp4'
    cmd_concat = (
        f"ffmpeg -y -f concat -safe 0 -i '{concat_txt}' -c copy '{combined_playing_local}'"
    )
    print('Combining playing clips into:', combined_playing_local)
    os.system(cmd_concat)

    if combined_playing_local.exists():
        combined_playing_drive = DRIVE_CLIPS_DIR / combined_playing_local.name
        try:
            shutil.copy2(combined_playing_local, combined_playing_drive)
            print('Copied combined playing video to Drive:', combined_playing_drive)
        except Exception as e:
            print('Failed to copy combined playing video to Drive:', e)

combined_nonplaying_local = None
combined_nonplaying_drive = None

if COMBINE_NONPLAYING_CLIPS and analysis_end_sec is not None:
    clips_sorted = clips_df.sort_values('start_sec') if not clips_df.empty else clips_df
    nonplay_segments = []
    current = 0.0

    # Only consider [0, analysis_end_sec]
    for _, row in clips_sorted.iterrows():
        s = float(row['start_sec'])
        e = float(row['end_sec'])

        if s >= analysis_end_sec:
            break

        s_clamped = max(0.0, min(s, analysis_end_sec))
        e_clamped = max(0.0, min(e, analysis_end_sec))

        if s_clamped > current:
            gap_start = current
            gap_end = s_clamped
            gap_dur = gap_end - gap_start
            if gap_dur >= MIN_NONPLAY_DURATION_SEC:
                nonplay_segments.append((gap_start, gap_end))

        current = max(current, e_clamped)
        if current >= analysis_end_sec:
            break

    if analysis_end_sec > current:
        gap_start = current
        gap_end = analysis_end_sec
        gap_dur = gap_end - gap_start
        if gap_dur >= MIN_NONPLAY_DURATION_SEC:
            nonplay_segments.append((gap_start, gap_end))

    print('Non-playing segments (within analysis window):', nonplay_segments)

    nonplay_clip_paths = []
    for idx, (ns, ne) in enumerate(nonplay_segments, start=1):
        start = max(0.0, ns)
        end = min(analysis_end_sec, ne)
        if end <= start:
            continue
        duration = end - start
        out_name = f'nonplay_{idx:03d}_{start:06.2f}-{end:06.2f}.mp4'
        out_path = CLIPS_OUTPUT_DIR / out_name
        cmd = (
            f"ffmpeg -y -ss {start:.3f} -i '{VIDEO_PATH}' -t {duration:.3f} "
            f"-c:v libx264 -preset veryfast -crf 18 -c:a aac -movflags +faststart '{out_path}'"
        )
        print('Cutting non-playing clip', idx, 'start=', start, 'end=', end)
        os.system(cmd)
        if out_path.exists():
            nonplay_clip_paths.append(out_path)
            try:
                shutil.copy2(out_path, DRIVE_CLIPS_DIR / out_name)
            except Exception as e:
                print('Failed to copy non-playing clip to Drive:', e)

    if nonplay_clip_paths:
        concat_txt_non = CLIPS_OUTPUT_DIR / 'nonplaying_concat_list.txt'
        with open(concat_txt_non, 'w') as f:
            for p in nonplay_clip_paths:
                f.write(f"file '{p}'\n")

        combined_nonplaying_local = CLIPS_OUTPUT_DIR / 'all_nonplaying.mp4'
        cmd_concat_non = (
            f"ffmpeg -y -f concat -safe 0 -i '{concat_txt_non}' -c copy '{combined_nonplaying_local}'"
        )
        print('Combining non-playing clips into:', combined_nonplaying_local)
        os.system(cmd_concat_non)

        if combined_nonplaying_local.exists():
            combined_nonplaying_drive = DRIVE_CLIPS_DIR / combined_nonplaying_local.name
            try:
                shutil.copy2(combined_nonplaying_local, combined_nonplaying_drive)
                print('Copied combined non-playing video to Drive:', combined_nonplaying_drive)
            except Exception as e:
                print('Failed to copy combined non-playing video to Drive:', e)

print('Done cutting and combining clips (where enabled).')

Video duration (sec): 163.29646666666667
Analysis end (sec): 163.2631
Cutting playing clip 1 start= 1.1849666666666667 end= 3.8866666666666667
Cutting playing clip 2 start= 7.591366666666667 end= 10.526633333333335
Cutting playing clip 3 start= 14.698466666666668 end= 16.933033333333334
Cutting playing clip 4 start= 18.30206666666667 end= 20.836933333333334
Cutting playing clip 5 start= 29.546633333333336 end= 32.1816
Cutting playing clip 6 start= 34.117866666666664 end= 36.35243333333333
Cutting playing clip 7 start= 38.622366666666665 end= 42.12486666666666
Cutting playing clip 8 start= 48.36543333333333 end= 50.433166666666665
Cutting playing clip 9 start= 50.0004 end= 52.16823333333333
Cutting playing clip 10 start= 61.54526666666666 end= 64.74746666666667
Cutting playing clip 11 start= 73.25696666666667 end= 75.32469999999999
Cutting playing clip 12 start= 76.99403333333333 end= 79.86256666666667
Cutting playing clip 13 start= 80.86456666666666 end= 85.70173333333334
Cutting playi

In [ ]:
# ====== SAVE METADATA ======
CLIPS_OUTPUT_DIR = OUTPUT_DIR / 'playing_clips_videos'
DRIVE_CLIPS_DIR = ORIGINAL_VIDEO_DIR / 'playing_clips_videos'

combined_playing_local = locals().get('combined_playing_local', None)
combined_playing_drive = locals().get('combined_playing_drive', None)
combined_nonplaying_local = locals().get('combined_nonplaying_local', None)
combined_nonplaying_drive = locals().get('combined_nonplaying_drive', None)

meta = {
    'version': VERSION,
    'original_video_path': str(ORIGINAL_VIDEO_PATH),
    'original_model_path': str(ORIGINAL_MODEL_PATH),
    'video_path': str(VIDEO_PATH),
    'model_path': str(MODEL_PATH),
    'fps': fps,
    'frame_count': frame_count,
    'width': width,
    'height': height,
    'ball_class_ids': list(BALL_CLASS_IDS),
    'conf_thres': CONF_THRES,
    'max_duration_seconds': MAX_DURATION_SECONDS,
    'max_frames_to_process': MAX_FRAMES_TO_PROCESS,
    'frame_stride': FRAME_STRIDE,
    'batch_size': BATCH_SIZE,
    'detection_steps': len(frame_indices),
    'debug_save_annotated': DEBUG_SAVE_ANNOTATED,
    'annotated_dir': str(ANNOTATED_DIR),
    'annotated_zip_path': annotated_zip_path,
    'ball_csv_local': str(signals_csv_local),
    'ball_csv_drive': str(BALL_CSV_DRIVE_PATH),
    'raw_csv_local': str(raw_csv_local) if os.path.exists(raw_csv_local) else None,
    'raw_csv_drive': str(ORIGINAL_VIDEO_DIR / raw_csv_local.name) if os.path.exists(raw_csv_local) else None,
    'motion_csv_local': str(motion_csv_local),
    'motion_csv_drive': str(motion_csv_drive),
    'playing_clips_csv_local': str(playing_clips_csv_local),
    'playing_clips_csv_drive': str(playing_clips_csv_drive),
    'playing_clips_video_dir_local': str(CLIPS_OUTPUT_DIR),
    'playing_clips_video_dir_drive': str(DRIVE_CLIPS_DIR),
    'move_dist_thresh': MOVE_DIST_THRESH,
    'static_grid_size': STATIC_GRID_SIZE,
    'static_window_sec': STATIC_WINDOW_SEC,
    'static_window_min_count': STATIC_WINDOW_MIN_COUNT,
    'conf_play_thresh': CONF_PLAY_THRESH,
    'min_clip_duration_sec': MIN_CLIP_DURATION_SEC,
    'max_gap_inside_sec': MAX_GAP_INSIDE_SEC,
    'pad_before_sec': PAD_BEFORE_SEC,
    'pad_after_sec': PAD_AFTER_SEC,
    'extra_cut_buffer_sec': EXTRA_CUT_BUFFER_SEC,
    'combine_playing_clips': COMBINE_PLAYING_CLIPS,
    'combine_nonplaying_clips': COMBINE_NONPLAYING_CLIPS,
    'min_nonplay_duration_sec': MIN_NONPLAY_DURATION_SEC,
    'analysis_end_sec': analysis_end_sec,
    'combined_playing_video_local': str(combined_playing_local) if combined_playing_local else None,
    'combined_playing_video_drive': str(combined_playing_drive) if combined_playing_drive else None,
    'combined_nonplaying_video_local': str(combined_nonplaying_local) if combined_nonplaying_local else None,
    'combined_nonplaying_video_drive': str(combined_nonplaying_drive) if combined_nonplaying_drive else None,
}

meta_json = OUTPUT_DIR / 'metadata.json'
with open(meta_json, 'w') as f:
    json.dump(meta, f, indent=2)

print('Saved metadata to:', meta_json)

Saved metadata to: /content/ball_signals/metadata.json
